In [1]:
import re
import pandas as pd


In [2]:
with open('night of knives vol1.txt', 'r', encoding='utf-8') as file:
    text_1 = file.read()


In [3]:

characters = ["Zato", "Millia", "?", "Venom", "Female Aristocrat", "Castle Guard F", "Gottfried", "Slayer", "Leon", "Super Soldier B",
              "Super Soldier A", "Super Soldier C", "Super Soldier D", "Super Soldier E", "Super Soldier F", "Castle Guard", "Soldier A",
               "Soldier B", "Soldier A+B", "God Series", "Johnny" ]

In [9]:

def make_i_o(text, characters = characters):
    
    used_characters = set()
    input_lines = {}
    output_lines = []
    last_character = None
    last_zato_line = None

    for line in text.split('\n'):
        found_character = False
    
        for char in characters:
            if char and re.match(f'^{re.escape(char)}', line):
                found_character = True
                if char == 'Zato':
                    output_lines.append(line)
                    last_zato_line = line
                else:
                    input_line = line[len(char)+1:].strip()
                    if last_zato_line:
                        input_line = f"{last_zato_line} {input_line}"
                        last_zato_line = None
                    if char not in input_lines:
                        input_lines[char] = []
                    input_lines[char].append(input_line)
                last_character = char
                break
                # elif last_character == 'Zato':
                #     input_lines[char].append((line[len(char)+1:].strip(), line))
                #     used_characters.add(char)
                # last_character = char
                # break
    
        if not found_character:
            if last_character == "Zato":
                output_lines.append(line)
            else:
                input_line = line.strip()
                if last_zato_line:
                    input_line = f"{last_zato_line} {input_line}"
                    last_zato_line = None
                if last_character not in input_lines:
                    input_lines[last_character] = []
                input_lines.append((last_character, input_line, line))
    zato_inputs = {}
    for char, lines in input_lines.items():
        if char != "Zato":
            for i in range(len(lines)):
                if i == len(lines) - 1:
                    zato_inputs[char] = {'input' : lines[i], 'output':''}
                else:
                    if 'Zato' in input_lines.keys():
                        index_zato = input_lines['Zato'].index(lines[i+1])
                        if index_zato == len(input_lines['Zato']) -1:
                            zato_inputs[char] = {'input':lines[i], 'output': input_lines['Zato'][index_zato]}
                        else:
                            zato_inputs[char] = {'input': lines[i], 'output': input_lines['Zato'][index_zato+1]}
    
    return zato_inputs


In [16]:
def format_text(text, characters=characters):
    used_characters = set()
    input_lines = []
    output_lines = []
    last_character = None
    last_zato_line = None
    
    for line in text.split('\n'):
        found_character = False
        
        for char in characters:
            if char and re.match(f'^{re.escape(char)}', line):
                found_character = True
                if char == 'Zato':
                    output_lines.append(line)
                    last_zato_line = line
                else:
                    input_line = line[len(char)+1:].strip()
                    if last_zato_line:
                        input_line = f"{last_zato_line} {input_line}"
                        last_zato_line = None
                    input_lines.append((char, input_line, line))
                last_character = char
                break
                
        if not found_character:
            if last_character == "Zato":
                output_lines.append(line)
            else:
                input_line = line.strip()
                if last_zato_line:
                    input_line = f"{last_zato_line} {input_line}"
                    last_zato_line = None
                input_lines.append((last_character, input_line, line))
                
    input_lines = [(char, in_line, out_line) for char, in_line, out_line in input_lines if char !='Zato' and char in characters]
    
    for char, in_line, out_line in input_lines:
        used_characters.add(char)
    
    zato_inputs = {}
    for char, in_line, out_line in input_lines:
        if char not in zato_inputs:
            zato_inputs[char] = []
        zato_inputs[char].append((in_line, out_line))
        
    data = []
    for char in used_characters:
        for in_line, out_line in zato_inputs[char]:
            data.append({"input": in_line, "output": out_line, "character": char})
    
    df = pd.DataFrame(data, columns=['input','output', 'character'])        
    return df

In [17]:
format_text(text_1)

,input,output,character
0,Muuguuff!,Super Soldier F Muuguuff!,Super Soldier F
1,,,Super Soldier F
2,Goohh!,Super Soldier A Goohh!,Super Soldier A
3,Zato It's a good thing I work best in the shad...,Super Soldier B Bruggh!,Super Soldier B
4,,,Super Soldier B
...,...,...,...
545,,,God Series
546,,,God Series
547,Zato Our founder. Guuuuuooooohhhhhgaaahhh!! *A...,Super Soldier D Guuuuuooooohhhhhgaaahhh!! *Ano...,Super Soldier D
548,Zato Out of the way!! *Zato mows him down with...,Super Soldier D Goohhwaaa!!,Super Soldier D


In [28]:
def separate_lines(text):
    character_pattern = re.compile(r'^[A-Za-z ]')
    lines = []
    current_line = ''
    
    for line in text.split('\n'):
        if character_pattern.match(line):
            if current_line:
                lines.append(current_line.strip())
                current_line = ''
            lines.append(line.strip())
        else:
            current_line += ' ' + line.strip()
    
    if current_line:
        lines.append(current_line.strip())
    lines = [x for x in lines if x != '']
    return lines

In [29]:
lines = separate_lines(text_1)

In [27]:
lines

['Millia Zato...where?',
 "Zato What's the matter?",
 'Zato In a sec...come in.',
 'Millia Sleepless night, sleep medication, and a body pillow, am I correct?',
 'Zato I love silky skin, guilty as charged.',
 "Millia If you wanted a doll wouldn't it be better to have a slave?",
 "They're much softer and more obedient, and their bodies can be quite the weapon if trained properly.",
 'Zato Relief could be obtained from embracing a doll...',
 'Millia And if I did, would it absolutely kill you?',
 "Zato If there was murderous intent in your embrace to kill me, dying wouldn't be too bad.",
 'Millia (Liar...)',
 'Millia Hey...what were you doing just now?',
 'Zato I was just counting something....Carrie...Trevor...Macalore...',
 'Millia What is that?',
 "Zato They're the names of dead members...my subordinates...",
 "Millia You're not possessed by dead Assassins are you? Don't joke around with something bothersome like that!",
 "Zato They were good men; I was fortunate to have them, but it's